# Lista de shapefiles de la clasificación de Guanajuato sentinel 2017 y 2018

Usar:

```
/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/ 
```

para 2017 y 2018 

y:

```
/LUSTRE/MADMEX/shapefiles/indi50k/indi50k_relacion_estados/Guanajuato_indi50k.csv
```

## 2017

/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/sentinel_2017/indi50k/

### 1 Encontrar lista de shapefiles de guanajuato:

csv=/LUSTRE/MADMEX/shapefiles/indi50k/indi50k_relacion_estados/Guanajuato_indi50k.csv

clasif=/LUSTRE/MADMEX/products/landcover/sentinel2/2017/indi50k/

```
for ind in $(sed 1d $csv);do ruta=$(find $clasif/$ind -name '*.shp');echo $ruta >> ~/lista_shapefiles_guanajuato_2017.txt;done
```

**Lo siguiente no es necesario ejecutar**

In [5]:
archivo = '/home/madmex_user/lista_shapefiles_guanajuato.txt'
#list_shapefiles = open("/home/madmex_user/lista_shapefiles_guanajuato.txt","r")

In [4]:
#list_shapefiles.close()

In [11]:
list_shapefiles=[]
with open(archivo) as src:
    list_shapefiles = src.read().splitlines()

In [13]:
list_shapefiles[0:2]

['/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/sentinel_2017/indi50k//F14C14/F14C14_madmex_sentinel2_2017.shp',
 '/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/sentinel_2017/indi50k//F14C13/F14C13_madmex_sentinel2_2017.shp']

#is this necessary ??:
#ogr2ogr -clipsrc /LUSTRE/MADMEX/shapefiles/shapes_estados_mexico_proyeccion_inegi_lcc/Guanajuato/Guanajuato.shp Guanajuato_sentinel_2017.shp #Guanajuato_merge.shp 

**FIN DE: Lo siguiente no es necesario ejecutar**

### 2.1 Un enfoque: 

**se requiere usar el `/home/madmex_user/shell merge_shapefiles.sh` y luego código de python (esto hay que modificar, mejor si sólo se usa python)**


```
cat merge_shapefiles.sh 
#!/bin/bash
# first parameter is list of shapefiles to merge
# second parameter is name of merge shapefile
# example: bash merge_shapefiles.sh lista_shapefiles_guanajuato.txt Guanajuato_merge

file="$2.shp"

for i in $(cat $1)
do
        if [ -f "$file" ]
        then
                echo "creating $2.shp" 
                ogr2ogr -f 'ESRI Shapefile' -update -append $file $i 
        else
                echo "merging..."
                ogr2ogr -f 'ESRI Shapefile' $file $i
        fi
done


```


Execute:


```
bash merge_shapefiles.sh lista_shapefiles_guanajuato.txt Guanajuato_merge
```

Then:

In [1]:
import os

import fiona
from fiona.crs import to_string

In [2]:
shapefile = os.path.expanduser('~') + '/Guanajuato_merge.shp'

In [3]:
shapefile

'/home/madmex_user/Guanajuato_merge.shp'

In [4]:
with fiona.open(shapefile) as src:
    schema = src.schema
    fc = [x for x in src]
    crs = to_string(src.crs)

In [5]:
schema

{'properties': OrderedDict([('id', 'int:18'),
              ('clave_31', 'int:18'),
              ('descr_31', 'str:80'),
              ('clave_17', 'int:18'),
              ('descr_17', 'str:80'),
              ('area', 'float:24.15')]),
 'geometry': 'Polygon'}

In [6]:
schema['geometry'] = ('Polygon', 'MultiPolygon')

In [7]:
filename =  os.path.expanduser('~') + '/'  + 'Guanajuato_merge' + '.gpkg'

In [8]:
filename

'/home/madmex_user/Guanajuato_merge.gpkg'

In [9]:
with fiona.open(filename, 
               'w',
               layer='Guanajuato_merge',
               schema = schema,
               crs = crs,
               driver='GPKG') as dst:
    [dst.write(x) for x in fc]

### 2.2 Otro enfoque: 

**más sencillo pero se genera un layer por cada archivo shapefile**

Execute: `/home/madmex_user/merge_shapefiles_to_geopackage.sh`

```
#!/bin/bash
# first parameter is list of shapefiles to merge
# second parameter is name of merge shapefile
# example: bash merge_shapefiles_to_geopackage.sh lista_shapefiles_guanajuato_2017.txt Guanajuato_merge

file="$2.gpkg"

for i in $(cat $1)
do
        if [ -f "$file" ]
        then
                echo "creating $2.gpkg" 
                ogr2ogr -f 'GPKG' -update -append $file $i 
        else
                echo "merging..."
                ogr2ogr -f 'GPKG' $file $i
        fi
done
```

```
bash merge_shapefiles_to_geopackage.sh lista_shapefiles_guanajuato_2017.txt Guanajuato_merge_gpkg
```

## 3 Resultados en:


1er enfoque: (2.1)

(se borró)

2o enfoque: (2.2)

```
/home/madmex_user/results/Guanajuato_sentinel2_2017.gpkg
```

O en sus rutas mapeadas a `LUSTRE`:

1er enfoque: (2.1)

(se borró)

2o enfoque: (2.2)

```
/LUSTRE/MADMEX/docker_antares3/conabio_cluster/home_madmex_user_conabio_docker_container_results/Guanajuato_sentinel2_2017.gpkg
```


## 2.3 Otra opcion:

a partir de una lista de shapefiles :

```
lista_shapefiles_guanajuato_2017.txt 
lista_shapefiles_guanajuato_2018.txt
```

se crea un solo geopackage co una sola layer sin usar ogr2ogr 

resultados en:

/home/madmex_user/results/Guanajuato_sentinel2_2017.gpkg 

/home/madmex_user/results/Guanajuato_sentinel2_2018.gpkg

In [2]:
import geopandas as gpd
import pandas as pd

In [3]:
inputfile = '/home/madmex_user/lista_shapefiles_guanajuato_2017.txt'
#inputfile = '/home/madmex_user/lista_shapefiles_chihuahua.txt'

In [4]:
with open(inputfile) as f:
    tilesfiles = f.read().splitlines()

In [5]:
len(tilesfiles)

52

In [6]:
tables = []
for tile in tilesfiles:
    tables.append(gpd.read_file(tile))

In [7]:
tables[0].tail()

,id,clave_31,descr_31,clave_17,descr_17,area,geometry
9565,9566,27,Pastizales,13,Pastizales y otra vegetación herbácea,3.193426,"POLYGON ((2616212.380 1082259.658, 2616222.380..."
9566,9567,27,Pastizales,13,Pastizales y otra vegetación herbácea,0.399312,"POLYGON ((2616492.380 1082179.658, 2616522.380..."
9567,9568,27,Pastizales,13,Pastizales y otra vegetación herbácea,0.021692,"POLYGON ((2618542.380 1082129.658, 2618562.380..."
9568,9569,3,Bosque de Encino y Bosque de Galería,2,Bosques latifoliados,0.062673,"POLYGON ((2621602.380 1082169.658, 2621622.380..."
9569,9570,14,Matorral Crasicaule,7,Matorral alto denso,33.940130,"MULTIPOLYGON (((2628132.380 1082719.658, 26281..."


In [8]:
df = pd.concat(tables).set_index('id').reset_index().sort_index()
tables = []

In [9]:
df.tail()

,id,clave_31,descr_31,clave_17,descr_17,area,geometry
359569,5538,29,Urbano y Construido,15,Urbano y construido,0.063153,"POLYGON ((2581492.406 861412.635, 2581502.406 ..."
359570,5539,28,Tierras Agrícolas,14,Tierras agrícolas,249.008783,"POLYGON ((2582542.406 863152.635, 2582552.406 ..."
359571,5540,29,Urbano y Construido,15,Urbano y construido,402.554899,"POLYGON ((2582972.406 863442.635, 2583032.406 ..."
359572,5541,29,Urbano y Construido,15,Urbano y construido,1.232971,"POLYGON ((2595892.406 861882.635, 2595902.406 ..."
359573,5542,28,Tierras Agrícolas,14,Tierras agrícolas,39922.445784,"MULTIPOLYGON (((2602362.406 885912.635, 260238..."


In [10]:
outputfile = '/home/madmex_user/Guanajuato_sentinel2_2017.gpkg'
#outputfile = '/home/madmex_user/Chihuahua_merge2.gpkg'

In [11]:
df.to_file(outputfile, driver='GPKG')

## 2.3 para lista de shapefiles de todo México

In [1]:
import glob
import os
import shutil

import geopandas as gpd
import pandas as pd

In [6]:
path = '/LUSTRE/MADMEX/products/landcover/sentinel2/'

### Año 2017

In [7]:
path_2017 = path + '2017/estados/'

In [8]:
lista_estados = os.listdir(path_2017)

In [9]:
lista_estados

['San_Luis_Potosi',
 'Mexico',
 'Chiapas',
 'Oaxaca',
 'Nayarit',
 'Colima',
 'Baja_California',
 'Hidalgo',
 'Veracruz_de_Ignacio_de_la_Llave',
 'Campeche',
 'Yucatan',
 'Sinaloa',
 'Guanajuato',
 'Chihuahua',
 'Durango',
 'Coahuila_de_Zaragoza',
 'Jalisco',
 'Tabasco',
 'Nuevo_Leon',
 'Baja_California_Sur',
 'Sonora',
 'Distrito_Federal',
 'Guerrero',
 'Tamaulipas',
 'Quintana_Roo',
 'Queretaro',
 'Aguascalientes',
 'Michoacan_de_Ocampo',
 'Puebla',
 'Morelos',
 'Tlaxcala',
 'Zacatecas']

In [ ]:
for k in range(len(lista_estados)):
    estado = lista_estados[k]
    print(estado)
    path_estado = path_2017 + estado + '/31/'
    lista_shapefiles_por_estado = glob.glob(path_estado + '/*.shp')
    tables = []
    for tile in lista_shapefiles_por_estado:
        tables.append(gpd.read_file(tile))
    df = pd.concat(tables).set_index('id').reset_index().sort_index()
    tables = []
    outputfile = os.path.expanduser('~') + '/geopackages/' + estado + '_sentinel2_2017.gpkg'
    df.to_file(outputfile, driver='GPKG')
    destiny_path = path_estado + estado + '_sentinel2_2017.gpkg'
    shutil.move(outputfile,destiny_path)
    df = None

San_Luis_Potosi
Mexico
Chiapas
Oaxaca
Nayarit
Colima
Baja_California
Hidalgo
Veracruz_de_Ignacio_de_la_Llave
Campeche
Yucatan
Sinaloa
Guanajuato
Chihuahua
Durango
Coahuila_de_Zaragoza
Jalisco
Tabasco
Nuevo_Leon
Baja_California_Sur
Sonora
Distrito_Federal
Guerrero
Tamaulipas
Quintana_Roo
Queretaro


### Año 2018

In [14]:
path_2018 = path + '2018/estados/'

In [15]:
lista_estados = os.listdir(path_2018)

In [16]:
lista_estados

['San_Luis_Potosi',
 'Mexico',
 'Chiapas',
 'Oaxaca',
 'Nayarit',
 'Colima',
 'Baja_California',
 'Hidalgo',
 'Veracruz_de_Ignacio_de_la_Llave',
 'Campeche',
 'Yucatan',
 'Sinaloa',
 'Guanajuato',
 'Chihuahua',
 'Durango',
 'Coahuila_de_Zaragoza',
 'Jalisco',
 'Tabasco',
 'Nuevo_Leon',
 'Baja_California_Sur',
 'Sonora',
 'Distrito_Federal',
 'Guerrero',
 'Tamaulipas',
 'Quintana_Roo',
 'Queretaro',
 'Aguascalientes',
 'Michoacan_de_Ocampo',
 'Puebla',
 'Morelos',
 'Tlaxcala',
 'Zacatecas']

In [ ]:
for k in range(len(lista_estados)):
    estado = lista_estados[k]
    print(estado)
    path_estado = path_2018 + estado + '/31/'
    lista_shapefiles_por_estado = glob.glob(path_estado + '/*.shp')
    tables = []
    for tile in lista_shapefiles_por_estado:
        tables.append(gpd.read_file(tile))
    df = pd.concat(tables).set_index('id').reset_index().sort_index()
    tables = []
    outputfile = os.path.expanduser('~') + '/geopackages/' + estado + '_sentinel2_2018.gpkg'
    df.to_file(outputfile, driver='GPKG')
    destiny_path = path_estado + estado + '_sentinel2_2018.gpkg'
    shutil.move(outputfile,destiny_path)
    df = None

San_Luis_Potosi
Mexico
Chiapas
Oaxaca
Nayarit
Colima
Baja_California
Hidalgo
Veracruz_de_Ignacio_de_la_Llave
Campeche
Yucatan
Sinaloa
Guanajuato
Chihuahua
Durango
Coahuila_de_Zaragoza
Jalisco
Tabasco
Nuevo_Leon
Baja_California_Sur
Sonora
Distrito_Federal
Guerrero
Tamaulipas
Quintana_Roo
Queretaro


#### Just some lines to help coding

In [17]:
path_estado = path_2018 + lista_estados[0] + '/31/'

In [18]:
path_estado

'/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/'

In [19]:
lista_shapefiles_por_estado = glob.glob(path_estado + '/*.shp')

In [20]:
lista_shapefiles_por_estado

['/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14A15_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14A25_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/G14C83_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14B82_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14A46_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14C37_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14D22_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14A21_madmex_sentinel2_2018.shp',
 '/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/F14C18_madmex_sent

In [21]:
tables = []
for tile in lista_shapefiles_por_estado:
    tables.append(gpd.read_file(tile))

In [24]:
tables[0].tail()

,id,clave_31,descr_31,clave_17,descr_17,area,geometry
10862,10863,4,Chaparral,8,Matorral mésico,0.026486,"POLYGON ((2618830.691 1274907.209, 2618840.691..."
10863,10864,2,Bosque de Coníferas: de Pino y Táscate,1,Bosques de aciculifolias y escuamifolias,23.214504,"MULTIPOLYGON (((2621160.691 1275437.209, 26211..."
10864,10865,3,Bosque de Encino y Bosque de Galería,2,Bosques latifoliados,3.703264,"POLYGON ((2622250.691 1275127.209, 2622300.691..."
10865,10866,20,Matorral Desértico Rosetófilo,9,Matorral bajo abierto,114.499912,"MULTIPOLYGON (((2622270.691 1276517.209, 26222..."
10866,10867,13,"Matorral Desértico Micrófilo, Mezquital Xerófi...",9,Matorral bajo abierto,177.078568,"MULTIPOLYGON (((2627560.691 1276397.209, 26275..."


In [25]:
df = pd.concat(tables).set_index('id').reset_index().sort_index()
tables = []

In [26]:
df.tail()

,id,clave_31,descr_31,clave_17,descr_17,area,geometry
438086,8760,14,Matorral Crasicaule,7,Matorral alto denso,0.888915,"MULTIPOLYGON (((2588219.606 1192127.662, 25882..."
438087,8761,28,Tierras Agrícolas,14,Tierras agrícolas,1906.630999,"MULTIPOLYGON (((2580159.606 1199467.662, 25801..."
438088,8762,14,Matorral Crasicaule,7,Matorral alto denso,0.065382,"POLYGON ((2589839.606 1192097.662, 2589849.606..."
438089,8763,28,Tierras Agrícolas,14,Tierras agrícolas,9.183421,"MULTIPOLYGON (((2590999.606 1192317.662, 25910..."
438090,8764,29,Urbano y Construido,15,Urbano y construido,9.350003,"POLYGON ((2594179.606 1192427.662, 2594199.606..."


In [21]:
estado = lista_estados[0]
outputfile = os.path.expanduser('~') + '/geopackages/' + estado + '_sentinel2_2018.gpkg'

In [22]:
outputfile

'/home/madmex_user/geopackages/San_Luis_Potosi_sentinel2_2018.gpkg'

In [34]:
df.to_file(outputfile, driver='GPKG')

In [23]:
destiny_path = path_estado + estado + '_sentinel2_2018.gpkg'

In [24]:
destiny_path

'/LUSTRE/MADMEX/products/landcover/sentinel2/2018/estados/San_Luis_Potosi/31/San_Luis_Potosi_sentinel2_2018.gpkg'

In [39]:
shutil.move(outputfile,destiny_path)

'/LUSTRE/MADMEX/products/landcover/sentinel2/2017/estados/San_Luis_Potosi/31/San_Luis_Potosi_sentinel2_2017.gpkg'